# Logistic Model

In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, roc_auc_score, auc, roc_curve, average_precision_score, precision_recall_curve
from sklearn.pipeline import Pipeline



#%matplotlib inline
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#pd.options.display.float_format = '{:.3f}'.format

Logistic REgressionda yapilacaklar
1- veriye baktim ve inceledim, boxplotlar cizdirdim
2- Outlier varmi yok mu diye, ve get_dummies lazimsa yapilir,
3- veriyi train ve test diye böleriz
4- Scaling yapariz

In [66]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.600,0.627,50,1
1,1,85,66,29,0,26.600,0.351,31,0
2,8,183,64,0,0,23.300,0.672,32,1
3,1,89,66,23,94,28.100,0.167,21,0
4,0,137,40,35,168,43.100,2.288,33,1


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [68]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.000,3.845,3.370,0.000,1.000,3.000,6.000,17.000
Glucose,768.000,120.895,31.973,0.000,99.000,117.000,140.250,199.000
BloodPressure,768.000,69.105,19.356,0.000,62.000,72.000,80.000,122.000
SkinThickness,768.000,20.536,15.952,0.000,0.000,23.000,32.000,99.000
Insulin,768.000,79.799,115.244,0.000,0.000,30.500,127.250,846.000
BMI,768.000,31.993,7.884,0.000,27.300,32.000,36.600,67.100
DiabetesPedigreeFunction,768.000,0.472,0.331,0.078,0.244,0.372,0.626,2.420
Age,768.000,33.241,11.760,21.000,24.000,29.000,41.000,81.000
Outcome,768.000,0.349,0.477,0.000,0.000,0.000,1.000,1.000


In [69]:
X=df.drop(["Outcome"], axis=1)
y=df["Outcome"]

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

# stratify traine hem 1 hem sifirdan esit gitsin istiyorum, cogunluk sifir olmamasini istiyorum,
#cünkü unbalans dataydi verimiz, traine dengeli bir veri göndermeye yariyor
# unbalans ile mücadele ederken, birde trainin unbalans olmasina izin vermiyoruz

#Popülasyonu, tabaka (strata) adı verilen homojen alt gruplara bölerek ve ardından 
# her alt grup içinde Basit Rastgele Örnekleme uygulayarak Tabakalı Örnekleme gerçekleştiririz.

# Scaling

In [72]:
## If you want,you can make Scaling,but than you may not make pipeline,because in pipeline There is Scaling oprerations

In [73]:
scaler = StandardScaler()

In [74]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  #teste fit yapmiyoruz,

# Finding with GridSEarch the Best Parameters

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
log_model = LogisticRegression(random_state=42)

#Pieline ile GridSearch yaptim,hata aldim, default model ile GridSearch yaptim

In [77]:
penalty = ["l1", "l2"]   # l1 lasso,l2 Ridge  2 hyper
C = np.logspace(-1, 5, 20)  # regularization yapiyor, C düstükce iyi yapiyor, alphanin ise yüksegi makbul  hyper
class_weight= ["balanced", None]    #kendisi datayi balans hale getiriyor,   hyper

solver = ["lbfgs", "liblinear", "sag", "saga"]  #hyper parametreler,data sayisi az ise liblinear seciliyor  hyper

param_grid = {"penalty" : penalty,
              "C" : C,
              "class_weight":class_weight,
            
              "solver":solver} 
               #paramgridi olusturuyorum asagida ekliyecem,hyperlari, icine verdim, o bunlaein optimum degerini verecek


grid_model = GridSearchCV(estimator=log_model,# logistic regression
                          param_grid=param_grid,
                          cv=10,
                          scoring = "accuracy", 
                          n_jobs = -1)

In [78]:
grid_model.fit(X_train_scaled,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': array([1.00000000e-01, 2.06913808e-01, 4.28133240e-01, 8.85866790e-01,
       1.83298071e+00, 3.79269019e+00, 7.84759970e+00, 1.62377674e+01,
       3.35981829e+01, 6.95192796e+01, 1.43844989e+02, 2.97635144e+02,
       6.15848211e+02, 1.27427499e+03, 2.63665090e+03, 5.45559478e+03,
       1.12883789e+04, 2.33572147e+04, 4.83293024e+04, 1.00000000e+05]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'liblinear', 'sag', 'saga']},
             scoring='accuracy')

In [79]:
grid_model.best_params_

{'C': 0.20691380811147897,
 'class_weight': None,
 'penalty': 'l2',
 'solver': 'lbfgs'}

# New Pipeline model with best Parameters

In [80]:
operations = [('scaler',MinMaxScaler()),('log_pipeline',LogisticRegression(class_weight= 'balanced',C=0.1,penalty='l2',solver='lbfgs',max_iter=10000,random_state=42))]

<IPython.core.display.Javascript object>

In [81]:
Pipeline(steps=operations)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('log_pipeline',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=10000, random_state=42))])

In [82]:
pipe_model = Pipeline(steps=operations)

In [83]:
pipe_model.fit(X_train,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('log_pipeline',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=10000, random_state=42))])

In [84]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred)) #bu o küseli parantez icindeki ciktilari veriyor
    print(classification_report(y_test, y_pred)) #büyük tablo gibi olani veriyor
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred)) 
    print(classification_report(y_train, y_train_pred))

In [85]:
eval_metric(pipe_model, X_train, y_train, X_test, y_test)

Test_Set
[[76 24]
 [17 37]]
              precision    recall  f1-score   support

           0       0.82      0.76      0.79       100
           1       0.61      0.69      0.64        54

    accuracy                           0.73       154
   macro avg       0.71      0.72      0.72       154
weighted avg       0.74      0.73      0.74       154


Train_Set
[[305  95]
 [ 65 149]]
              precision    recall  f1-score   support

           0       0.82      0.76      0.79       400
           1       0.61      0.70      0.65       214

    accuracy                           0.74       614
   macro avg       0.72      0.73      0.72       614
weighted avg       0.75      0.74      0.74       614



In [86]:
scores = cross_validate(pipe_model, X_train, y_train, scoring = ['precision','recall','f1','accuracy'], cv = 10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores.mean()[2:]

#DataPreprocessing'te bir kac islemi yapmadigim icin sonuclar kötü cikti

test_precision   0.610
test_recall      0.682
test_f1          0.637
test_accuracy    0.731
dtype: float64

# Finding the best threshold value

In [87]:
y_pred_proba = pipe_model.predict_proba(X_train)
y_pred_proba

array([[0.64872182, 0.35127818],
       [0.48761703, 0.51238297],
       [0.58472579, 0.41527421],
       ...,
       [0.59385632, 0.40614368],
       [0.49666322, 0.50333678],
       [0.46879701, 0.53120299]])

In [88]:
fp_rate,tp_rate,thresholds = roc_curve(y_train, y_pred_proba[:,1])

#that are the parameteters, so we could find our best threshold value.
#fp_rate: False pozitif rate
#tp_rate: true pozitif rate

![](2022-10-24-10-01-05.png)

In [89]:
optimal_idx = np.argmax(tp_rate - fp_rate)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

0.4823124660496318

### 2.way to find threshold value

Finding Best Threshold for the most balanced score between recall and precision

In [90]:
pipe_model.predict_proba(X_test)[:,1]     #[:,1] if I dont write that.I get the both proba value for 0 and 1

array([0.55991305, 0.4826106 , 0.47625785, 0.47759333, 0.22129105,
       0.44303013, 0.5522023 , 0.67910849, 0.36783998, 0.58139317,
       0.43180717, 0.43172935, 0.40530394, 0.44266269, 0.4182239 ,
       0.48463026, 0.63205036, 0.37624831, 0.60169961, 0.39757677,
       0.48883714, 0.72771516, 0.4378215 , 0.7118496 , 0.56211772,
       0.32007896, 0.58933909, 0.34898774, 0.49271457, 0.33557473,
       0.34543785, 0.33735586, 0.49907089, 0.58672971, 0.61519114,
       0.37487586, 0.47310709, 0.35673987, 0.5411513 , 0.60573822,
       0.56341125, 0.38504855, 0.42304649, 0.45619159, 0.42192483,
       0.53846205, 0.43416184, 0.3897672 , 0.5970555 , 0.43275233,
       0.5661111 , 0.49644602, 0.4947067 , 0.35949184, 0.56227187,
       0.42273505, 0.59334702, 0.44233988, 0.62921599, 0.43327025,
       0.60579486, 0.43028022, 0.23909798, 0.57380693, 0.33939777,
       0.44535182, 0.58228454, 0.31512436, 0.47682789, 0.60132566,
       0.45113563, 0.40088031, 0.47187862, 0.5896929 , 0.32763

In [91]:
y_pred2 = pd.Series(pipe_model.predict_proba(X_test)[:,1]).apply(lambda x: 1 if x>= optimal_threshold else 0)
y_pred2

0      1
1      1
2      0
3      0
4      0
      ..
149    0
150    0
151    0
152    1
153    0
Length: 154, dtype: int64

In [92]:
y_train_pred2 = pd.Series(pipe_model.predict_proba(X_train)[:,1]).apply(lambda x: 1 if x>= optimal_threshold else 0)
y_train_pred2

0      0
1      1
2      0
3      0
4      0
      ..
609    0
610    0
611    0
612    1
613    1
Length: 614, dtype: int64

In [93]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    #y_train_pred = model.predict(X_train)
    #y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred2)) #bu o köseli parantez icindeki ciktilari veriyor
    print(classification_report(y_test, y_pred2)) #büyük tablo gibi olani veriyor
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred2)) 
    print(classification_report(y_train, y_train_pred2))



    #bu eval metric benim yeni pred2 degerlerini ölcmek icin kullandim, y_pred yazan yerleri y_pred2 olarak degistirdim

In [94]:
eval_metric(pipe_model, X_train,y_train,X_test,y_test)

Test_Set
[[71 29]
 [13 41]]
              precision    recall  f1-score   support

           0       0.85      0.71      0.77       100
           1       0.59      0.76      0.66        54

    accuracy                           0.73       154
   macro avg       0.72      0.73      0.72       154
weighted avg       0.75      0.73      0.73       154


Train_Set
[[284 116]
 [ 47 167]]
              precision    recall  f1-score   support

           0       0.86      0.71      0.78       400
           1       0.59      0.78      0.67       214

    accuracy                           0.73       614
   macro avg       0.72      0.75      0.72       614
weighted avg       0.76      0.73      0.74       614

